<a href="https://colab.research.google.com/github/KendallScott/ML2/blob/main/Scott_Kendall_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2

This is called DeathToGridSearch because with this example you will never have to think about how to manage a large number of classifiers etc simultaneously.  You will now be able to run and collect results in a very straightforward manner.  #LongLongLiveGridSearch!

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from itertools import product
from sklearn.model_selection import StratifiedKFold, cross_val_predict, cross_val_score

import itertools
from itertools import *
from numpy import arange
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset?resource=download

df = pd.read_csv('/content/drive/MyDrive/Employee.csv', encoding = 'ISO-8859-1',low_memory=False)

Mounted at /content/drive


In [4]:
df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [5]:
X = df.loc[:,df.columns != 'LeaveOrNot']
y = df['LeaveOrNot'].values.flatten()

In [6]:
df_num = X.select_dtypes(exclude='object')
num_cols = df_num.columns

# impute numeric with median
for i in num_cols:
    df_num[i] = df_num[i].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))
df_num.info()

df_cat = X.select_dtypes(include='object')
cat_cols = df_cat.columns

# scale numerical data
scaler = StandardScaler()
df_num = scaler.fit_transform(df_num)

#one hot encode cat data
encoder = OneHotEncoder(sparse_output=False)
df_cat = encoder.fit_transform(df_cat)
X = np.hstack([df_num,df_cat])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   JoiningYear                4653 non-null   int64
 1   PaymentTier                4653 non-null   int64
 2   Age                        4653 non-null   int64
 3   ExperienceInCurrentDomain  4653 non-null   int64
dtypes: int64(4)
memory usage: 145.5 KB


In [7]:
education = df[['Education', 'LeaveOrNot','City']].groupby(['Education', 'LeaveOrNot']).count().reset_index()
education

,Education,LeaveOrNot,City
0,Bachelors,0,2472
1,Bachelors,1,1129
2,Masters,0,447
3,Masters,1,426
4,PHD,0,134
5,PHD,1,45


### Dictionary functions

In [8]:
import re
#reference: https://stackoverflow.com/questions/69300562/how-to-define-the-grid-for-using-grid-search-from-scratch-in-python

#create the dictionary used to iterate and run grid search
def makeGrid(pars_dict):
    keys=pars_dict.keys()
    length = len(keys)
    combinations=itertools.product(*pars_dict.values())
    ds=[dict(zip(keys,cc)) for cc in combinations]
    return ds


In [25]:
def dict_cleanup(param_dict, new_dict):
  new_data = {}

  for  v in param_dict.keys():
      new = re.sub('[^a-zA-Z_]', '', v)
      def slicedict(d, s):
        return {k:v for k,v in d.items() if k.startswith(new)}
      param_dict_copy = slicedict(param_dict, new)
      new_data_values = list(param_dict_copy.values())
      new_data_empty = {}
      new_data_empty[new]=new_data_values
      new_data.update(new_data_empty)
  return new_data

### Grid Search Function

In [10]:
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X,y)

def search_results(model_type, model, grid, results):
  grid=grid
  model_type=model_type
  parameters = {}
  param_dict = {}

  for x, i in enumerate(grid):
            model.set_params(**i)
            model.fit(X,y)
            score = cross_val_score(model, X, y, scoring='accuracy', cv=5)
            score=score.mean()
            results =np.append(results, np.ravel(score))
            if max(results)==score:
              #print('best model & params: ', i)
              best_model = i
              best_score = score

            #updating dictionary with results

            param_dict.update(i)
            iteration = x
            param_dict.update(param_dict)
            param_dict["results"] = score
            param_dict["model"] = model_type
            param_dict["iteration"] = iteration

            keys=param_dict.keys()
            new_dict = {}

            #updating columns to be unique for each iteration, to prevent replacing values
            for x, i in enumerate(keys):
                key =i + str(iteration)
                new_dict[key]=param_dict[i]

                parameters.update(new_dict)

                #new_dict[key] = param_dict[i]

  return parameters

### Running the custom grid search and dictionary functions

In [11]:
def grid_search(model_version):
  param_res_clean = {}
  model_version = model_version
  model_selection = ['Logistic Regression', 'Random Forest', 'Linear SVC', 'XGBoost']
  #creating different params for different models in loop
  model_selection_results = {}
  grid_parameters = {}
  grid_results = np.empty(0)

  if model_version=='Logistic Regression':
    model=LogisticRegression(random_state=45,penalty='elasticnet',class_weight='balanced',solver='saga')
    l1 = arange(0, 1, 0.1)
    C = [  0.001,  0.1,  0.5, 1]
    max_iter = [25,  50, 60, 75, 100]

    hyper_dict = {'C': C, 'l1_ratio': l1, 'max_iter':max_iter}
    grid = makeGrid(hyper_dict)
    param = {}
    param_res =search_results(model_version, model, grid, grid_results)
    return param_res

  elif model_version=='Random Forest':
    model = RandomForestClassifier(criterion='gini',class_weight='balanced')
    n_estimators = [50, 100, 200]
    max_features = ['sqrt','log2',20]

    hyper_dict = {'n_estimators': n_estimators, 'max_features': max_features}
    grid = makeGrid(hyper_dict)
    param_res = {}
    param_res =search_results(model_version, model, grid, grid_results)
    return param_res

  elif model_version=='Linear SVC':
    model = LinearSVC(max_iter=100000,tol=0.01,dual=True,class_weight = 'balanced')
    C = [.001,.005,.01,.05,.1,.2,.5,.8,1,3,5,20,50]

    hyper_dict = {'C': C}
    grid = makeGrid(hyper_dict)
    param_res = {}
    param_res =search_results(model_version, model, grid, grid_results)
    return param_res

In [12]:
Linear_SVC = grid_search('Linear SVC')
Linear_SVC

{'C0': 0.001,
 'results0': 0.6883792430384718,
 'model0': 'Linear SVC',
 'iteration0': 0,
 'C1': 0.005,
 'results1': 0.6888077336197637,
 'model1': 'Linear SVC',
 'iteration1': 1,
 'C2': 0.01,
 'results2': 0.6885947587863669,
 'model2': 'Linear SVC',
 'iteration2': 2,
 'C3': 0.05,
 'results3': 0.6916027395678135,
 'model3': 'Linear SVC',
 'iteration3': 3,
 'C4': 0.1,
 'results4': 0.691172863033159,
 'model4': 'Linear SVC',
 'iteration4': 4,
 'C5': 0.2,
 'results5': 0.6905283947195178,
 'model5': 'Linear SVC',
 'iteration5': 5,
 'C6': 0.5,
 'results6': 0.690313340956077,
 'model6': 'Linear SVC',
 'iteration6': 6,
 'C7': 0.8,
 'results7': 0.690313340956077,
 'model7': 'Linear SVC',
 'iteration7': 7,
 'C8': 1,
 'results8': 0.690313340956077,
 'model8': 'Linear SVC',
 'iteration8': 8,
 'C9': 3,
 'results9': 0.690313340956077,
 'model9': 'Linear SVC',
 'iteration9': 9,
 'C10': 5,
 'results10': 0.690313340956077,
 'model10': 'Linear SVC',
 'iteration10': 10,
 'C11': 20,
 'results11': 0.69031

In [13]:
Logistic_Regression = grid_search('Logistic Regression')
Logistic_Regression

{'C0': 0.001,
 'l1_ratio0': 0.0,
 'max_iter0': 25,
 'results0': 0.6873042052134946,
 'model0': 'Logistic Regression',
 'iteration0': 0,
 'C1': 0.001,
 'l1_ratio1': 0.0,
 'max_iter1': 50,
 'results1': 0.6873042052134946,
 'model1': 'Logistic Regression',
 'iteration1': 1,
 'C2': 0.001,
 'l1_ratio2': 0.0,
 'max_iter2': 60,
 'results2': 0.6873042052134946,
 'model2': 'Logistic Regression',
 'iteration2': 2,
 'C3': 0.001,
 'l1_ratio3': 0.0,
 'max_iter3': 75,
 'results3': 0.6873042052134946,
 'model3': 'Logistic Regression',
 'iteration3': 3,
 'C4': 0.001,
 'l1_ratio4': 0.0,
 'max_iter4': 100,
 'results4': 0.6873042052134946,
 'model4': 'Logistic Regression',
 'iteration4': 4,
 'C5': 0.001,
 'l1_ratio5': 0.1,
 'max_iter5': 25,
 'results5': 0.7072873427809154,
 'model5': 'Logistic Regression',
 'iteration5': 5,
 'C6': 0.001,
 'l1_ratio6': 0.1,
 'max_iter6': 50,
 'results6': 0.7072873427809154,
 'model6': 'Logistic Regression',
 'iteration6': 6,
 'C7': 0.001,
 'l1_ratio7': 0.1,
 'max_iter7': 

In [14]:
Random_Forest = grid_search('Random Forest')
Random_Forest

{'n_estimators0': 50,
 'max_features0': 'sqrt',
 'results0': 0.8252721665915942,
 'model0': 'Random Forest',
 'iteration0': 0,
 'n_estimators1': 50,
 'max_features1': 'log2',
 'results1': 0.8229081921393344,
 'model1': 'Random Forest',
 'iteration1': 1,
 'n_estimators2': 50,
 'max_features2': 20,
 'results2': 0.8216190245198248,
 'model2': 'Random Forest',
 'iteration2': 2,
 'n_estimators3': 100,
 'max_features3': 'sqrt',
 'results3': 0.8241968977743899,
 'model3': 'Random Forest',
 'iteration3': 3,
 'n_estimators4': 100,
 'max_features4': 'log2',
 'results4': 0.8231216289571858,
 'model4': 'Random Forest',
 'iteration4': 4,
 'n_estimators5': 100,
 'max_features5': 20,
 'results5': 0.8198983634200709,
 'model5': 'Random Forest',
 'iteration5': 5,
 'n_estimators6': 200,
 'max_features6': 'sqrt',
 'results6': 0.8241964357899357,
 'model6': 'Random Forest',
 'iteration6': 6,
 'n_estimators7': 200,
 'max_features7': 'log2',
 'results7': 0.8226931383758936,
 'model7': 'Random Forest',
 'ite

In [ ]:
objective = ['binary:logistic']
eval_metric = ['error']
learning_rates = [0.1, 0.2]
max_depths = [3, 5,7, 10, 20]
gamma = [0,1,3]
lambda_ls = [1,2,3]
alpha = [0,0.1,1]

hyper_dict = {'objective': objective, 'eval_metric': eval_metric, 'eta': learning_rates, 'max_depth': max_depths, 'gamma': gamma, 'lambda': lambda_ls, 'alpha': alpha}
grid = makeGrid(hyper_dict)

for ids, (train_index, test_index) in enumerate(skf.split(X, y)):
    dtrain = xgb.DMatrix(X[train_index], label=y[train_index])
    dtest = xgb.DMatrix(X[test_index], label=y[test_index])
    evallist = [(dtrain, 'train'), (dtest, 'eval')]

parameter_results = {}
grid_results = np.empty(0)
for x, i in enumerate(grid):
    print(i)
    score = xgb.cv(params=i, num_boost_round=200, dtrain=dtrain,  nfold=3, stratified=True, early_stopping_rounds=3, verbose_eval=1)
    index=score.shape[0]-1
    score=1-(score.iloc[index,2])
    print('current score: ',score)
    grid_results = np.insert(grid_results,[0], score)

    #updating dictionary with results
    param_dict = {}
    param_dict.update(i)
    iteration = x
    param_dict.update(param_dict)
    param_dict["results"] = score
    param_dict["model"] = "XGBoost"
    param_dict["iteration"] = iteration

    keys=param_dict.keys()
    new_dict = {}

    for x, z in enumerate(keys):
      key =z + str(iteration)
      new_dict[key]=param_dict[z]
      parameter_results.update(new_dict)

{'objective': 'binary:logistic', 'eval_metric': 'error', 'eta': 0.1, 'max_depth': 3, 'gamma': 0, 'lambda': 1, 'alpha': 0}
[0]	train-error:0.21003+0.00562	test-error:0.21051+0.01123
[1]	train-error:0.21067+0.00536	test-error:0.20986+0.01097
[2]	train-error:0.21003+0.00562	test-error:0.21051+0.01123
[3]	train-error:0.20825+0.00458	test-error:0.20825+0.01239
[4]	train-error:0.21003+0.00562	test-error:0.21051+0.01123
[5]	train-error:0.20406+0.00584	test-error:0.20503+0.01231
[6]	train-error:0.20068+0.00890	test-error:0.20471+0.00925
[7]	train-error:0.20293+0.00571	test-error:0.20664+0.01196
[8]	train-error:0.19955+0.00345	test-error:0.20664+0.01277
current score:  0.7952933591231464
{'objective': 'binary:logistic', 'eval_metric': 'error', 'eta': 0.1, 'max_depth': 3, 'gamma': 0, 'lambda': 1, 'alpha': 0.1}
[0]	train-error:0.21003+0.00562	test-error:0.21051+0.01123
[1]	train-error:0.21067+0.00536	test-error:0.20986+0.01097
[2]	train-error:0.21003+0.00562	test-error:0.21051+0.01123
[3]	train-e

In [ ]:
new_data = {}
new_data_empty = {}
new_dict = {}

for  v in parameter_results.keys():
      new = re.sub('[^a-zA-Z_]', '', v)
      new = str(new)
      def slicedict(d, s):
        return {k:v for k,v in d.items() if k.startswith(new)}
      param_dict_copy = slicedict(param_dict, new)
      new_data_values = list(param_dict.values())

      new_data[new]=new_data_values
      print(new_data)

In [52]:
parameter_results_clean = dict_cleanup(parameter_results)

{}
{}


AttributeError: ignored

In [24]:
parameter_results

{'objective0': 'binary:logistic',
 'eval_metric0': 'error',
 'eta0': 0.1,
 'max_depth0': 3,
 'gamma0': 0,
 'lambda0': 1,
 'alpha0': 0,
 'results0': 0.7952933591231464,
 'model0': 'XGBoost',
 'iteration0': 0,
 'objective1': 'binary:logistic',
 'eval_metric1': 'error',
 'eta1': 0.1,
 'max_depth1': 3,
 'gamma1': 0,
 'lambda1': 1,
 'alpha1': 0.1,
 'results1': 0.7952933591231464,
 'model1': 'XGBoost',
 'iteration1': 1,
 'objective2': 'binary:logistic',
 'eval_metric2': 'error',
 'eta2': 0.1,
 'max_depth2': 3,
 'gamma2': 0,
 'lambda2': 1,
 'alpha2': 1,
 'results2': 0.7952933591231464,
 'model2': 'XGBoost',
 'iteration2': 2,
 'objective3': 'binary:logistic',
 'eval_metric3': 'error',
 'eta3': 0.1,
 'max_depth3': 3,
 'gamma3': 0,
 'lambda3': 2,
 'alpha3': 0,
 'results3': 0.7952933591231464,
 'model3': 'XGBoost',
 'iteration3': 3,
 'objective4': 'binary:logistic',
 'eval_metric4': 'error',
 'eta4': 0.1,
 'max_depth4': 3,
 'gamma4': 0,
 'lambda4': 2,
 'alpha4': 0.1,
 'results4': 0.79529335912314